# deploy with python

### 1.使用python部署*.so文件

In [1]:
# 使用python部署*.so文件
import onnx
import time
import tvm
import numpy as np
import tvm.relay as relay
from PIL import Image
from tvm.contrib import graph_runtime

libpath = "../models/mobilenet_pc.so"
graph_json_path = "../models/mobilenet_pc.json"
param_path = "../models/mobilenet_pc.params"

# 接下来我们加载导出的模型去测试导出的模型是否可以正常工作
loaded_json = open(graph_json_path).read()
loaded_lib = tvm.runtime.load_module(libpath)
loaded_params = bytearray(open(param_path, "rb").read())

# 这里执行的平台为CPU
ctx = tvm.cpu()


# 以下的图片读取仅仅是为了测试
mean = [123., 117., 104.]                   # 在ImageNet上训练数据集的mean和std
std = [58.395, 57.12, 57.375]

def transform_image(image):                # 定义转化函数，将PIL格式的图像转化为格式维度的numpy格式数组
    image = image - np.array(mean)
    image /= np.array(std)
    image = np.array(image).transpose((2, 0, 1))
    image = image[np.newaxis, :].astype('float32')
    return image

img = Image.open('../imgs/plane.png').resize((224, 224)) # 这里我们将图像resize为特定大小
x = transform_image(img)



module = graph_runtime.create(loaded_json, loaded_lib, ctx)
module.load_params(loaded_params)
module.set_input("input.1", x)
module.run()
out_deploy = module.get_output(0).asnumpy()
print(type(out_deploy))
print(out_deploy.argmax())
# print(out_deploy)

# 输出tvm运行的时间
since = time.time()
for i in range(4000):
    module.run()
time_elapsed = time.time() - since
print('Time elapsed is {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

<class 'numpy.ndarray'>
404
Time elapsed is 0m 35s
